In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('../data/processed/Cleaned_Dataframe.csv')

Remove columns that do not add value to classifying text

In [3]:
df = df.drop('Unnamed: 0', 1)
df = df.drop('Dataline', 1)
df = df.drop('PlayerLinenumber', 1)

Generate Cateogires to predict Player

In [4]:
df.Play = pd.Categorical(df.Play)
df.Player = pd.Categorical(df.Player)
df.PlayerLine = pd.Categorical(df.PlayerLine)

In [5]:
df['Play_code'] = df.Play.cat.codes
df['Player_code'] = df.Player.cat.codes
df['PlayerLine_code'] = df.PlayerLine.cat.codes

In [6]:
df = df.dropna()

In [7]:
features = ['Play_code','PlayerLine_code', 'Act', 'Scene', 'Line']
x = np.array(df[features])
y = np.array(df['Player_code'])

Split data for training and testing, using 30% of the data for testing. Set the random_sate to 1 so the tests can be rerun for same results.

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)

The code below is kept in the notebook for reference sake but should not be run.

Gonna be honest here, didn't think Logistic Regression would be good for this dataset. Went ahead and tried it since it was a simple model to use but it appears to have failed with the given layout.

model_lg = LogisticRegression(verbose=2)
model_lg.fit(x_train, y_train)
y_pred = model_lg.predict(x_test)

Running a RandomForest classifier model. The outcome is fairly negligible considering the True/False Negative/Positive ratios tend to be around 50% which could be the equivalent of just tossing a coin on each entry and hoping for the best. The weighted average indicates a 66% rate which seems decent but this could be corrupt due to the large reduction in the datasets classifications from trimming the lower 3/4 quantiles in processing

In [9]:
model_rf = RandomForestClassifier(verbose=2)
model_rf.fit(x_train, y_train)
y_pred = model_rf.predict(x_test)
print(metrics.classification_report(y_test, y_pred))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


              precision    recall  f1-score   support

           2       0.00      0.00      0.00         2
           3       0.78      0.90      0.84        91
           4       0.33      0.50      0.40         2
           5       0.50      0.22      0.31         9
           6       1.00      0.33      0.50         3
           7       0.47      0.54      0.50        52
           8       0.76      0.70      0.73        23
           9       0.50      0.50      0.50         2
          10       0.68      0.59      0.63        80
          11       0.67      0.63      0.65        82
          12       0.84      0.91      0.88        54
          13       0.67      0.94      0.78        17
          14       0.00      0.00      0.00         5
          15       0.62      0.70      0.66        40
          16       0.50      0.40      0.44         5
          17       0.75      0.65      0.70        60
          18       0.35      0.35      0.35        17
          19       0.37    

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   31.7s finished
D:\Workspace\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Workspace\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The importance of the Line indicates to me that the model is utilizing the fact that there is a higher probability that a player would have multiple sequences of lines. Using this, for the same Act-Scene a sequence of lines has a higher chances of being the same player, i.e., Act 1 Scene 1 Line 5 is known to be Romeo then predicting Act 1 Scene 1 Line 6 would lend weight to the Player being Romeo

In [10]:
feature_df = pd.DataFrame({'Importance':model_rf.feature_importances_, 'Features': features })
print(feature_df)

   Importance         Features
0    0.227643        Play_code
1    0.212381  PlayerLine_code
2    0.085406              Act
3    0.087152            Scene
4    0.387417             Line


In [ ]:
model_svc = SVC(verbose=2)
model_svc.fit(x_train, y_train)
y_pred = model_svc.predict(x_test)
print(metrics.classification_report(y_test, y_pred))

[LibSVM]